In [1]:
import requests
import os
from dotenv import load_dotenv
import qdrant_client
from qdrant_client.models import Filter, FieldCondition, Range, MatchValue
import psycopg2
import csv
import pandas as pd

In [2]:
load_dotenv()

api_key = os.environ.get('API_KEY')
api_url = os.environ.get('API_URL')
qdrant_url = os.environ.get('PYTHON_QDRANT_URL')
qdrant_api_key = os.environ.get('QDRANT_API_KEY')

In [3]:
def send_search_request(query_str):
    request_body = {
        "query": query_str,
        "page": 1
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.post(api_url + "/search", json=request_body, headers=headers)
    return response.json()

# results = send_search_request('Software engineer programmer starts to realize the magic system is akin to programming')
# for result in results:
#     index = result['DocEmbedding']["index"]
#     story_id = result['DocEmbedding']["story_id"]
#     link = f'https://www.royalroad.com/fiction/{story_id}'
#     print(link, index)

In [4]:
def send_search_request_document_group(query_str, group_size):
    request_body = {
        "doc_group_size": group_size,
        "query": query_str,
        "page": 1
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.post(api_url + "/search", json=request_body, headers=headers)
    return response.json()

In [3]:
def create_document_group(group_size):
    request_body = {
        "doc_group_size": group_size
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.post(api_url + "/document_group", json=request_body, headers=headers)
    return response.status_code

In [4]:
# create_document_group(50)

204

In [7]:
def index_document_group(story_id, group_size):
    request_body = {
        "story_id": story_id,
        "doc_group_size": group_size
    }

    headers = {"Content-Type": "application/json", "Authorization": api_key}
    response = requests.put(api_url + "/document_group", json=request_body, headers=headers)
    return response.status_code

In [8]:
index_document_group(57018, 50)

204

In [38]:
# results = send_search_request_document_group('Software engineer programmer starts to realize the magic system is akin to programming', 50)

# for result in results:
#     story_id = result['DocGroupEmbedding']["story_id"]
#     index = result['DocGroupEmbedding']["index"]
#     link = f'https://www.royalroad.com/fiction/{story_id}'
#     print(link, index)

https://www.royalroad.com/fiction/13143 0
https://www.royalroad.com/fiction/26163 0
https://www.royalroad.com/fiction/28995 0
https://www.royalroad.com/fiction/24 0
https://www.royalroad.com/fiction/20177 0
https://www.royalroad.com/fiction/23074 0
https://www.royalroad.com/fiction/16878 0


In [32]:
# Replace with your PostgreSQL connection URL
db_url = os.environ.get('DATABASE_URL')

# SQL query to select the columns you want
query = "SELECT DISTINCT story_id FROM doc_embeddings"

# File path for the single CSV file
output_file_path = "distinct_story_ids.csv"

# Batch size
batch_size = 10000

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(db_url)

# Create a cursor
cursor = conn.cursor()

# Execute the query
cursor.execute(query)

# Open the CSV file for writing
with open(output_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write header
    csv_writer.writerow(["story_id"])

    batch_number = 0
    while True:
        results = cursor.fetchmany(batch_size)
        if not results:
            break

        # Write data for the current batch
        csv_writer.writerows(results)

        batch_number += 1

# Close the cursor and the connection
cursor.close()
conn.close()

print(f"Data has been exported to {output_file_path}")

Data has been exported to distinct_story_ids.csv


In [54]:
distinct_story_ids = pd.read_csv('distinct_story_ids.csv')
distinct_story_ids

,story_id
0,44127
1,26264
2,37876
3,55864
4,56903
...,...
69807,67482
69808,74710
69809,63618
69810,46401


In [55]:
import pandas as pd

# Assuming you have a DataFrame named distinct_story_ids

# Calculate the number of rows in each chunk
chunk_size = len(distinct_story_ids) // 20

# Create a list to store the 20 data chunks
data_chunks = []

# Split the DataFrame into 20 chunks
for i in range(20):
    if i < 19:
        chunk = distinct_story_ids.iloc[i * chunk_size:(i + 1) * chunk_size]
    else:
        # For the last chunk, include any remaining rows
        chunk = distinct_story_ids.iloc[i * chunk_size:]

    data_chunks.append(chunk)

# Now, data_chunks contains 20 DataFrames, each representing a chunk of the original DataFrame.


In [56]:
import os

# Assuming you have already split the DataFrame into data_chunks as shown in the previous answer

# Create a directory to save the chunks
output_directory = "./distinct_story_id_chunks"
os.makedirs(output_directory, exist_ok=True)

# Save each chunk to a numbered CSV file
for i, chunk in enumerate(data_chunks):
    chunk.to_csv(os.path.join(output_directory, f"chunk_{i + 1}.csv"), index=False)

print("Data chunks saved to the directory:", output_directory)


Data chunks saved to the directory: ./distinct_story_id_chunks


In [3]:
svd_client = qdrant_client.QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

/home/skeptrune/.local/lib/python3.10/site-packages/qdrant_client/qdrant_remote.py:115: UserWarning: Api key is used with unsecure connection.
  warnings.warn("Api key is used with unsecure connection.")


In [4]:
svd_client.count(collection_name="doc_embeddings")

CountResult(count=1226222)

In [6]:
svd_client.count(collection_name="doc_group_50")

CountResult(count=11066)

In [53]:
# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(db_url)

# Create a cursor
cursor = conn.cursor()

output_file_path = "erred_story_ids.csv"

with open(output_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["story_id"])

    for index, row in distinct_story_ids.iterrows():
        # SQL query to select the count of rows for the current story_id
        query = f"SELECT COUNT(*) FROM doc_embeddings WHERE story_id = {row['story_id']}"
        cursor.execute(query)
        result = cursor.fetchone()
        num_pg_docs = int(result[0])

        # qdrant scroll to get all embeddings for the current story_id
        qdrant_points = svd_client.scroll(
            collection_name="doc_embeddings",
            scroll_filter=Filter(
                must=[
                    FieldCondition(
                        key='story_id',
                        match=MatchValue(value=str(row['story_id']))
                    )
                ]
            ),
            limit = num_pg_docs
        )
        num_qdrant_points = len(qdrant_points[0])
        if num_qdrant_points < num_pg_docs:
            csv_writer.writerow([row['story_id']])
            print(row['story_id'], num_pg_docs, num_qdrant_points)

cursor.close()
conn.close()

65863 2 1
37974 87 86
59073 4 0
46910 2 0
38334 2 1
2952 1 0
35528 3 1
12165 2 0
46889 14 0
65771 46 36
11152 14 8
55205 2 0
3311 38 0
3495 6 0
38832 22 7
38284 3 2
46804 26 24
36082 1 0
38111 374 373
35473 9 6
2865 2 1
12311 21 0
46896 221 0
35412 10 6
47007 1 0
3543 2 0
65855 7 5
34399 2 0
55216 55 0
55059 8 0
9297 1 0
9298 9 0
46878 6 0
71563 13 0
65797 5 4
63875 2 0
61989 1 0
63825 6 0
3587 85 2
12194 1 0
65830 54 40
3084 7 0
59054 2 0
64036 75 21
37840 15 14
3468 43 0
66400 102 21
63853 2 0
25747 1 0
37500 8 7
55308 87 0
38264 4 2
36344 8 2
46325 29 21
55038 65 0
19762 1 0
36081 2 0
44795 12 10
3031 10 0
46979 2 0
11709 14 3
45569 35 0
45551 5 0
31490 2 0
38837 8 1
11652 6 1
3112 6 0
50779 1 0
55120 27 0
46387 154 123
12024 378 5
2977 2 0
36059 1 0
36312 3 2
63831 4 0
63932 12 0
46270 31 20
12318 47 0
65057 1 0
11661 8 4
3420 9 0
12303 1 0
3304 4 0
56255 36 0
54931 1 0
22689 41 3
38827 25 7
3312 1 0
36014 1 0
9326 5 0
11206 23 4
9295 5 0
3443 4 0
38853 2 0
62028 1 0
45070 70 69
44

ResponseHandlingException: timed out